## Collecting LoL Posts from Reddit
Gulliver Wutz

In [2]:
import praw
import sys
import time
import datetime
import pytz
import pandas as pd

def unix_to_cest(unix_timestamp):
    dt_utc = datetime.datetime.utcfromtimestamp(unix_timestamp)
    cest = pytz.timezone('Europe/Berlin')
    dt_cest = dt_utc.replace(tzinfo=pytz.utc).astimezone(cest)
    formatted_dt = dt_cest.strftime('%Y-%m-%d %H:%M:%S')
    return formatted_dt

pd.set_option('display.max_colwidth', 20)

In [2]:
reddit = praw.Reddit(
    client_id = "YOUR_ID",
    client_secret = "YOUR_SECRET",
    user_agent = "YOUR_APP_NAME"
)

In [3]:
# Set up temporary dataframe
columns = [
    "Subreddit", "Title", "ID", "Full ID", "Author", "URL", "Is Self-post", 
    "Score", "Comment Count",
    "Created CEST", "Collected CEST", "Updated CEST", "Content"
]
temp_df = pd.DataFrame(columns=columns)

subreddit_names = ["summonerschool", "leagueoflegends", "LeagueofLegendsMeta", "LeaguePBE",
                   "MarksmanMains", "supportlol", "AatroxMains", "AhriMains", "akalimains",
                   "AkshanMains", "alistarmains", "amumumains", "AniviaMains", "AnnieMains",
                   "ApheliosMains", "AsheMains", "Aurelion_Sol_mains", "azirmains", "bardmains",
                   "BelVethMains", "blitzcrankmains", "BrandMains", "BraumMains", "Caitlynmains",
                   "CamilleMains", "CassiopeiaMains", "CorkiMains", "Dariusmains", "DianaMains",
                   "DirtySionMains", "Draven", "DrMundoMains", "ekkomains", "ezrealmains",
                   "Elisemains", "EvelynnMains", "FiddlesticksMains", "FioraMains", "fizzmains",
                   "galiomains", "gangplankmains", "GarenMains", "GnarMains", "GragasMains",
                   "GravesMains", "GwenMains", "HecarimMains", "HeimerdingerMains", "Illaoi",
                   "IreliaMains", "ivernmains", "Janna", "JarvanIVmains", "Jaxmains",
                   "jaycemains", "JhinMains", "kaisamains", "KalistaMains", "karmamains",
                   "karthusmains", "KassadinMains", "KatarinaMains", "Kaylemains", "KaynMains",
                   "Kennenmains", "KhaZixMains", "Kindred", "KledMains", "KogMawMains",
                   "KSanteMains", "leagueofjinx", "LeBlancMains", "LeeSinMains", "LilliaMains",
                   "LissandraMains", "LucianMains", "lulumains", "lux", "malphitemains",
                   "MalzaharMains", "MaokaiMains", "MilioMains", "MissFortuneMains",
                   "MordekaiserMains", "MorganaMains", "NamiMains", "nasusmains",
                   "NautilusMains", "neekomains", "NidaleeMains", "NilahMains", "nocturnemains",
                   "nunumains", "Olafmains", "OriannaMains", "ornnmains", "PantheonMains",
                   "PoppyMains", "pykemains", "QiyanaMains", "QuinnMains", "ChoGathMains",
                   "RakanMains", "RammusMains", "reksaimains", "RellMains", "RenataMains",
                   "RenektonMains", "Rengarmains", "Rivenmains", "Rumblemains", "RyzeMains",
                   "SamiraMains", "sejuanimains", "sennamains", "SeraphineMains", "settmains",
                   "shacomains", "Shen", "shyvanamains", "Sivir", "SkarnerMains", "sonamains",
                   "SorakaMains", "SwainMains", "sylasmains", "syndramains", "Talonmains",
                   "Tahmkenchmains", "taricmains", "TeemoTalk", "thesecretweapon", "ThreshMains",
                   "TristanaMains", "Trundlemains", "TryndamereMains", "TwistedFateMains",
                   "TwitchMains", "Udyrmains", "UrgotMains", "Velkoz", "vexmains", "ViMains",
                   "viktormains", "VladimirMains", "VolibearMains", "warwickmains",
                   "Wukongmains", "xayahmains", "XerathMains", "XinZhaoMains", "YasuoMains",
                   "YIMO", "YoneMains", "yorickmains", "yuumimains", "vaynemains",
                   "VarusMains", "VeigarMains", "ViegoMains", "zedmains", "ZeriMains",
                   "ZiggsMains", "ZileanMains", "zoemains", "zyramains"]

iter = 0

for subreddit_name in subreddit_names:
    iter += 1
    print("Subreddit " + str(iter) + ":")
    print(subreddit_name)
    subreddit = reddit.subreddit(subreddit_name)

    # Get the current Unix timestamp
    current_timestamp = int(time.time())
    new_posts = subreddit.new(limit=1000)

    for post in new_posts:
        new_data = {
        'Subreddit': subreddit_name,
        'Title': post.title,
        'ID': post.id,
        'Full ID': post.name,
        'Author': post.author,
        'URL': post.url,
        'Is Self-post': post.is_self,
        'Score': post.score,
        'Comment Count': post.num_comments,
        'Created CEST': unix_to_cest(post.created_utc),
        'Collected CEST': unix_to_cest(current_timestamp),
        'Updated CEST': unix_to_cest(current_timestamp),
        'Content': post.selftext
        }

        new_row = pd.DataFrame([new_data])
        temp_df = pd.concat([temp_df, new_row], ignore_index=True)

    print("temp_df shape: " + str(temp_df.shape))
    print("\n")



# Load CSV conatining all collected posts
posts_df = pd.read_csv('reddit_posts.csv')
print("Dataset size before: " + str(posts_df.shape))



# Update Upvotes and Comment Counts existsing posts
posts_df_merged = posts_df.merge(temp_df[['Full ID', 'Score', 'Comment Count', 'Updated CEST']], on='Full ID', how='left', suffixes=('', '_new'))
mask = ((posts_df_merged['Score'] != posts_df_merged['Score_new']) | 
        (posts_df_merged['Comment Count'] != posts_df_merged['Comment Count_new'])) & posts_df_merged['Score_new'].notna()
columns_to_update = ['Score', 'Comment Count', 'Updated CEST']
for col in columns_to_update:
    posts_df.loc[mask, col] = posts_df_merged.loc[mask, f'{col}_new']
print("Updated " + str(mask.sum()) + " posts in dataset!")



# Add all new found posts 
ids_not_in_posts_df = ~temp_df['Full ID'].isin(posts_df['Full ID'])
temp2_df = temp_df[ids_not_in_posts_df]
posts_df = pd.concat([posts_df, temp2_df], ignore_index=True)
print("Added " + str(ids_not_in_posts_df.sum()) + " new posts to dataset!")
print("Dataset size after: " + str(posts_df.shape))
print("\n")



# Consistency Check
#posts_df = posts_df.dropna(subset=['Author'])
if posts_df['ID'].is_unique:
    print("Post IDs are unique!")
else:
    print("Post IDs NOT are unique!")

if posts_df['Full ID'].is_unique:
    print("Post Full IDs are unique!")
else:
    print("Post Full IDs NOT are unique!")

if posts_df.isna().any().any():
    print(str(posts_df.isna().sum().sum()) + " NaN-Values detected!")
else:
    print("No NaN-Values detected!")


# Write updated dataset to CSV
posts_df.to_csv('reddit_posts.csv', index=False)
posts_df

Subreddit 1:
summonerschool
temp_df shape: (987, 13)


Subreddit 2:
leagueoflegends
temp_df shape: (1968, 13)


Subreddit 3:
LeagueofLegendsMeta
temp_df shape: (2920, 13)


Subreddit 4:
LeaguePBE
temp_df shape: (3917, 13)


Subreddit 5:
MarksmanMains
temp_df shape: (4908, 13)


Subreddit 6:
supportlol
temp_df shape: (5896, 13)


Subreddit 7:
AatroxMains
temp_df shape: (6882, 13)


Subreddit 8:
AhriMains
temp_df shape: (7867, 13)


Subreddit 9:
akalimains
temp_df shape: (8855, 13)


Subreddit 10:
AkshanMains
temp_df shape: (9848, 13)


Subreddit 11:
alistarmains
temp_df shape: (10826, 13)


Subreddit 12:
amumumains
temp_df shape: (11806, 13)


Subreddit 13:
AniviaMains
temp_df shape: (12801, 13)


Subreddit 14:
AnnieMains
temp_df shape: (13779, 13)


Subreddit 15:
ApheliosMains
temp_df shape: (14778, 13)


Subreddit 16:
AsheMains
temp_df shape: (15768, 13)


Subreddit 17:
Aurelion_Sol_mains
temp_df shape: (16767, 13)


Subreddit 18:
azirmains
temp_df shape: (17759, 13)


Subreddit 19:
b

,Subreddit,Title,ID,Full ID,Author,URL,Is Self-post,Score,Comment Count,Created CEST,Collected CEST,Updated CEST,Content
0,summonerschool,how do i win aga...,1c64jj9,t3_1c64jj9,LumineLover420,https://www.redd...,True,30,58,2024-04-17 10:21:30,2024-04-17 12:07:36,2024-05-26 10:03:02,i get fed in lan...
1,summonerschool,48 Laws of Power,1c63pix,t3_1c63pix,ImamAhmad11,https://www.redd...,True,3,4,2024-04-17 09:22:47,2024-04-17 12:07:36,2024-04-17 13:27:03,"Hello, \n\n \nA..."
2,summonerschool,How to Think?,1c62eq6,t3_1c62eq6,CheapAuguries,https://www.redd...,True,1,12,2024-04-17 07:57:58,2024-04-17 12:07:36,2024-04-27 12:28:49,I’ve been playin...
3,summonerschool,Advice for new j...,1c603ro,t3_1c603ro,Ok_Sheepherder5038,https://www.redd...,True,11,10,2024-04-17 05:45:35,2024-04-17 12:07:36,2024-05-26 10:03:02,Ive taken a liki...
4,summonerschool,How to convince ...,1c5wt2g,t3_1c5wt2g,MinatoShrimp,https://www.redd...,True,0,8,2024-04-17 03:02:01,2024-04-17 12:07:36,2024-04-27 12:28:49,Hi guys. For con...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
180602,zyramains,New item: Blackf...,1chbbfy,t3_1chbbfy,banggu_,https://i.redd.i...,False,47,28,2024-05-01 04:42:38,2024-05-26 13:22:45,2024-05-26 13:22:45,I dont have mana...
180603,zyramains,Blood Moon Zyra ...,1ch0bij,t3_1ch0bij,Regular-Poet-3657,https://www.redd...,False,20,5,2024-04-30 20:43:11,2024-05-26 13:22:45,2024-05-26 13:22:45,https://www.arts...
180604,zyramains,Winx Club Zyra F...,1cgw8hj,t3_1cgw8hj,LordksOP,https://youtu.be...,False,15,3,2024-04-30 17:52:36,2024-05-26 13:22:45,2024-05-26 13:22:45,
180605,zyramains,Blood Moon Zyra ...,1cgklgv,t3_1cgklgv,Regular-Poet-3657,https://www.redd...,False,20,0,2024-04-30 06:58:11,2024-05-26 13:22:45,2024-05-26 13:22:45,https://twitter....
